# Week7 Fama-MacBeth回归

## 分工：
石宛青：主要负责主体部分；王梦涵：主要负责习题部分。共同讨论遇到的问题。


## 介绍：
前文我们掌握了组合分析：优点：非参数；缺点：较难包含多个控制变量

Fama & MacBeth （1973）允许控制较多其他变量，线性假设；被广泛用于实证资产定价研究。

## 目的：
复现Fama和French（1993）三因素模型（beta,size,bm）的FM回归。

## 理论：
两步法：从本质上讲，两步 Fama-MacBeth 回归利用了预期回报与风险因素暴露之间的线性关系

#### step0：计算个股在每个时点的风险暴露敞口beta

#### step1：使用暴露（特征）作为解释变量进行横截面回归

<center>$r_{i,t+1} = \alpha_i+\lambda_t^M\beta_{i,t}^M+\lambda_t^{SMB}\beta_{i,t}^{SMB}+\lambda_t^{HML}\beta_{i,t}^{HML}+\epsilon_{i,t}$



#### step2：对横截面回归结果进行时间序列平均,手动计算T值
<center>$\frac{1}{T}\sum_{t=1}^{T}\hat{\lambda_t^f}$


## 实证内容：
### 1.数据准备

（1）导入股票收益月度数据（crsp_monthly）的超额收益、第三周计算的beta

（2）导入公司财务数据（compustat），以计算bm,size

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import statsmodels.api as sm


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
####导入数据

tidy_finance = sqlite3.connect(database='data/tidy_finance_python.sqlite')#data文件夹下有一个sqlite文件
crsp_monthly = pd.read_sql_query(sql='SELECT permno, gvkey, month, ret_excess, mktcap, mktcap_lag FROM crsp_monthly',
                                 con=tidy_finance,parse_dates={'month'})## 这里比原文多导入了滞后市值，用于练习加权，3326353行
compustat = pd.read_sql_query(sql = 'SELECT datadate, gvkey, be FROM compustat',con=tidy_finance,parse_dates={'datadate'})#549426行
##提前将week3算出的beta导进sql，目前已经导入共享Dropbox了
beta = pd.read_sql_query(sql = 'SELECT month, permno, beta_monthly FROM beta',con=tidy_finance,parse_dates={'month'})#3336322行

（3）计算账面市值比bm : $be/mktcap$  和规模size : $ln(mktcap)$

In [4]:
# 处理公司特征数据

compustat['month'] = pd.to_datetime(compustat['datadate']).dt.to_period('M').dt.to_timestamp()
#r语言中向下取整，python可以转为月份格式再取时间戳得到相同功能

characteristics = pd.merge(compustat, crsp_monthly, on=['gvkey', 'month'], how='left')#股票收益数据集合并到公司特征集
characteristics = pd.merge(characteristics, beta, on=['permno', 'month'], how='left')#再和beta合并
characteristics['bm'] = characteristics['be'] / characteristics['mktcap']#得到bm
characteristics['log_mktcap'] = np.log(characteristics['mktcap'])#计算得到size
characteristics['sorting_date'] = characteristics['month'] + pd.DateOffset(months=6)
#因为年报最晚6个月后公布，因此为避免前瞻性偏误，设定一个延后6个月的日期，在该日期时才能使用对应数据

characteristics = characteristics[["gvkey", "bm", "log_mktcap","beta_monthly", "sorting_date"]].rename(columns={"beta_monthly": "beta"})
characteristics

,gvkey,bm,log_mktcap,beta,sorting_date
0,011358,NaN,NaN,NaN,1960-07-01
1,002796,NaN,NaN,NaN,1960-07-01
2,007068,NaN,NaN,NaN,1960-07-01
3,006303,NaN,NaN,NaN,1960-07-01
4,005410,NaN,NaN,NaN,1960-07-01
...,...,...,...,...,...
549421,104833,NaN,NaN,NaN,2023-06-01
549422,183907,NaN,NaN,NaN,2023-06-01
549423,137611,NaN,NaN,NaN,2023-06-01
549424,037527,NaN,NaN,NaN,2023-06-01


 （4）生成Fama-MacBeth数据:

 'ret_excess_lead', 'beta',  'bm','log_mktcap'

In [5]:
# 生成Fama-MacBeth数据

#根据sorting_date将公司特征集合并到股票收益集
data_fama_macbeth = pd.merge(crsp_monthly, characteristics, left_on=['gvkey', 'month'], right_on=['gvkey', 'sorting_date'], how='left')

data_fama_macbeth = data_fama_macbeth.sort_values(by=['permno', 'month'])

#将前一年的数据空值填充
data_fama_macbeth[['beta', 'bm', 'log_mktcap']] = data_fama_macbeth.groupby("permno")[['beta', 'bm', 'log_mktcap']].fillna(method='ffill')

#t月的beta,bm,size对应t+1月的收益率
crsp_monthly_lead = (crsp_monthly.loc[:, ['permno', 'month', 'ret_excess']]
                    .assign(month=lambda x: x['month'] - pd.DateOffset(months=1))).rename(columns={"ret_excess": "ret_excess_lead"})
data_fama_macbeth = pd.merge(data_fama_macbeth,crsp_monthly_lead,on=['permno', 'month'],how='left')

data_fama_macbeth = data_fama_macbeth[['permno', 'month', 'ret_excess_lead', 'beta',  'bm','log_mktcap','mktcap_lag']].dropna()
#问题：这里的beta由于先和公司特征匹配再填充，损失了很多值？使用原始beta做稳健性发现beta系数变大，但依然十分不显著
data_fama_macbeth

,permno,month,ret_excess_lead,beta,bm,log_mktcap,mktcap_lag
74,10001.0,1990-12-01,0.007958,0.081100,0.938894,2.307796,10.048500
75,10001.0,1991-01-01,0.008187,0.081100,0.938894,2.307796,10.013000
76,10001.0,1991-02-01,-0.015938,0.081100,0.938894,2.307796,10.144750
77,10001.0,1991-03-01,0.034174,0.081100,0.938894,2.307796,10.276500
78,10001.0,1991-04-01,-0.004700,0.081100,0.938894,2.307796,10.013000
...,...,...,...,...,...,...,...
3326347,93436.0,2022-07-01,-0.074389,1.993018,0.027663,13.903696,701030.202209
3326348,93436.0,2022-08-01,-0.039489,1.993018,0.027663,13.903696,931110.623250
3326349,93436.0,2022-09-01,-0.144468,1.993018,0.027663,13.903696,863615.620800
3326350,93436.0,2022-10-01,-0.147226,1.993018,0.027663,13.903696,837659.500000


In [6]:
data_fama_macbeth_copy=data_fama_macbeth ## 这里加一个copy，方便后面练习二三使用

## 2. FM回归
### step1 横截面回归
在每个月进行OLS横截面回归

In [7]:
import statsmodels.api as sm
def fm1_reg(data):
    formula = 'ret_excess_lead ~ beta + bm + log_mktcap'
    model = sm.OLS.from_formula(formula, data=data)
    return model.fit().params

# 对每个月份应用回归
risk_premiums = data_fama_macbeth.groupby('month').apply(fm1_reg).reset_index(drop = 'True')
risk_premiums

,Intercept,beta,bm,log_mktcap
0,0.041592,-0.034635,0.027157,-0.001710
1,-0.013122,0.002502,0.007309,0.005275
2,-0.079722,0.029901,0.023318,0.008490
3,0.086708,-0.038654,-0.032397,-0.003647
4,-0.027339,0.011859,0.009438,0.000382
...,...,...,...,...
696,0.059798,0.007046,-0.003936,-0.010263
697,-0.137040,-0.019313,-0.003850,0.007235
698,-0.080569,0.029282,0.039173,0.014634
699,-0.097512,0.002875,0.004902,0.015787


### step2 计算时序均值
随后使用mean/标准误得到t值

In [8]:
summary_df = pd.DataFrame({'mean_values': risk_premiums.mean()*100, 't_values': (risk_premiums.mean() / risk_premiums.std()) * (len(risk_premiums)**0.5)})
summary_df

,mean_values,t_values
Intercept,1.215502,4.771331
beta,0.005146,0.049929
bm,0.150568,3.222465
log_mktcap,-0.104206,-2.941909


进行NW调整，设为滞后6阶。与r结果不一样的原因是r使用了“NeweyWest(.)”，没有设置滞后阶数而是根据Bartlett方法估计，且进行了prewhite处理。

若r使用NeweyWest(., lag = 6, prewhite = FALSE)则可得到与以下python代码一样的结果

原理：https://blog.csdn.net/weixin_40628687/article/details/88955575

In [9]:
summary_df['NW_t']  = [sm.OLS.from_formula(f'{factor}  ~ 1', data=risk_premiums).fit(cov_type="HAC", cov_kwds={'maxlags': 6}).tvalues[0] for factor in risk_premiums.columns]
summary_df

,mean_values,t_values,NW_t
Intercept,1.215502,4.771331,4.125457
beta,0.005146,0.049929,0.045422
bm,0.150568,3.222465,2.809913
log_mktcap,-0.104206,-2.941909,-2.768976


## 练习

In [10]:
## 为横截面回归以及数据整个编写一个函数，方便练习使用_石宛青
def run_fama_macbeth_swq(data, formula, weight=0):
    if weight == 0:
        risk_premiums = data.groupby('month').apply(lambda x:sm.OLS.from_formula(formula, data=x).fit().params).reset_index(drop=True)
    else:
        risk_premiums = data.groupby('month').apply(lambda x:sm.WLS.from_formula(formula, data=x,weights=x['mktcap_lag']).fit().params).reset_index(drop=True)
   
    summary_df = pd.DataFrame({
        'mean_values': risk_premiums.mean() * 100,
        't_values': (risk_premiums.mean() / risk_premiums.std()) * (len(risk_premiums) ** 0.5)})

    # NW_t values
    summary_df['NW_t'] = [sm.OLS.from_formula(f'{factor}  ~ 1', data=risk_premiums).fit(cov_type="HAC", cov_kwds={'maxlags': 6}).tvalues[0] for factor in risk_premiums.columns]

    return summary_df

result = run_fama_macbeth_swq(data_fama_macbeth,'ret_excess_lead ~ beta + bm + log_mktcap')
result

,mean_values,t_values,NW_t
Intercept,1.215502,4.771331,4.125457
beta,0.005146,0.049929,0.045422
bm,0.150568,3.222465,2.809913
log_mktcap,-0.104206,-2.941909,-2.768976


In [11]:
## 为横截面回归以及数据整个编写一个函数，方便练习使用_王梦涵
def csr_tsagg(data,formula,weight):
    if weight == 0:
        risk_premiums  = (data
                          .groupby("month",group_keys=False)
                          .apply(lambda x: sm.OLS.from_formula(formula=formula, data=x).fit().params)  
                          .reset_index())
    else:
        risk_premiums = (data_fama_macbeth
                         .groupby("month", group_keys=False)
                         .apply(lambda x: sm.WLS.from_formula(formula=formula, data=x, weights=x['mktcap_lag']).fit().params)
                         .reset_index())
    
    risk_premiums = (risk_premiums
                 .melt(id_vars="month",
                       var_name="factor",
                       value_name="estimate"))
    price_of_risk= (risk_premiums
                    .groupby("factor",group_keys=False)
                    .aggregate(risk_premium =("estimate", lambda x: np.mean(x)*100),
                               t_statistic = ("estimate", lambda x: np.mean(x)/np.std(x)*len(x)**0.5)
                               )
                    .reset_index())
    
    price_of_risk_newey_west = (risk_premiums
                                .groupby("factor",group_keys=False)
                                .apply(lambda x: x.assign(t_statistic_newey_west = sm.OLS.from_formula(formula="estimate~1", data=x).fit().tvalues[0])
                                                        .tail(1))
                                .get(["factor","t_statistic_newey_west"])
                                .reset_index(drop=True))
    
    result = (price_of_risk
              .merge(price_of_risk_newey_west,
                     how="left",
                     on=["factor"]))
    
    return result

In [12]:
csr_tsagg(data_fama_macbeth,formula="ret_excess_lead ~ beta + log_mktcap + bm",weight=0)

,factor,risk_premium,t_statistic,t_statistic_newey_west
0,Intercept,1.215502,4.774738,4.125457
1,beta,0.005146,0.049965,0.045422
2,bm,0.150568,3.224766,2.809913
3,log_mktcap,-0.104206,-2.944009,-2.768976


### 练习一

本节我们使用个股作为资产来估计风险溢价，接下来我们从 Kenneth French主页下载一个资产的样本，尝试使用行业投资组合来计算风险溢价

##### 选择我们之前下载过的十行业


In [13]:
industries_ff_monthly = pd.read_sql_query("SELECT * FROM industries_ff_monthly",
                                          con=tidy_finance,
                                          parse_dates={"month"})

In [14]:
## 宽数据转化为长数据方便后续操作
industries_ff_monthly = (industries_ff_monthly
 .melt(id_vars="month",
       var_name = "industry",
       value_name="ret"
       ))

In [15]:
factors_ff3_monthly = pd.read_sql_query(sql="SELECT * FROM factors_ff3_monthly",
                                        con=tidy_finance,
                                        parse_dates={"month"})

In [16]:
industries_ff_monthly = (industries_ff_monthly
                        .merge(factors_ff3_monthly,
                               how="left",
                               on="month")
                        .assign(ret_excess=lambda x: (x["ret"]-x["rf"]))
                        .drop(["ret","rf"],axis=1))

In [17]:
industries_ff_monthly

,month,industry,mkt_excess,smb,hml,ret_excess
0,1960-01-01,nodur,-0.0698,0.0209,0.0278,-0.0369
1,1960-02-01,nodur,0.0117,0.0051,-0.0193,0.0264
2,1960-03-01,nodur,-0.0163,-0.0049,-0.0294,-0.0176
3,1960-04-01,nodur,-0.0171,0.0032,-0.0228,0.0134
4,1960-05-01,nodur,0.0312,0.0121,-0.0370,0.0624
...,...,...,...,...,...,...
7555,2022-08-01,other,-0.0377,0.0140,0.0029,-0.0281
7556,2022-09-01,other,-0.0935,-0.0081,0.0005,-0.0824
7557,2022-10-01,other,0.0783,0.0006,0.0801,0.1143
7558,2022-11-01,other,0.0460,-0.0352,0.0138,0.0570


####  计算三个因子的风险暴露（使用滚动回归计算）


编写滚动回归函数，相比beta那一节的函数增加了两个参数，一个是回归方程的类型，一个是资产类别（股票或者行业组合）

In [18]:
from statsmodels.regression.rolling import RollingOLS

In [19]:
def roll_capm_estimation(data, window_size, min_obs,formula,params,asset_type):
    data=data.sort_values(["month"])

    betas = (RollingOLS.from_formula(formula = formula, 
                                    data = data, 
                                    window = window_size,
                                    min_nobs=min_obs)
             .fit()
             .params[params])
    
    return pd.DataFrame({
        'month': data['month'],
        asset_type : data[asset_type],
        'beta': betas})

1. 市场因子

In [20]:
beta_mkt = (industries_ff_monthly
              .groupby(["industry"],group_keys=False)
              .apply(roll_capm_estimation,window_size = 60, min_obs = 48,formula="ret_excess~mkt_excess",params="mkt_excess",asset_type="industry")
              .dropna()
              .rename(columns={"beta": "beta_mkt"})
              )              

2. 规模因子

In [21]:
beta_smb = (industries_ff_monthly
              .groupby(["industry"],group_keys=False)
              .apply(roll_capm_estimation,window_size = 60, min_obs = 48,formula="ret_excess~smb",params="smb",asset_type="industry")
              .dropna()
              .rename(columns={"beta": "beta_smb"})
              )              

3. 价值因子

In [22]:
beta_hml = (industries_ff_monthly
              .groupby(["industry"],group_keys=False)
              .apply(roll_capm_estimation,window_size = 60, min_obs = 48,formula="ret_excess~hml",params="hml",asset_type="industry")
              .dropna()
              .rename(columns={"beta": "beta_hml"})
              )             

In [23]:
## 合并风险暴露beta
beta_factors = (beta_mkt
                .merge(beta_smb,
                       how="left",
                       on=["month","industry"])
                .merge(beta_hml,
                       how="left",
                       on=["month","industry"])
                .assign(month = lambda x: x["month"]+pd.DateOffset(months=1)))  ## 因为滚动回归将数据结果放在窗口的最后一期，所以将beta滞后一期，避免出现下面第三题提到的前瞻性偏差

In [24]:
data_fama_macbeth  = (industries_ff_monthly
                           .get(["month","industry","ret_excess"])
                           .rename(columns={"ret_excess":"ret_excess_lead"})
                           .assign(month=lambda x: x["month"]-pd.DateOffset(months=1))     ## 处理ret_excess为ret_excess_lead
                           .merge(beta_factors,
                                  how="left",
                                  on=["month","industry"])
                           .dropna())     ## 因为滚动窗口回归，beta很多NA值

##### 进行FM回归

In [25]:
csr_tsagg(data_fama_macbeth,formula="ret_excess_lead ~ beta_mkt + beta_smb + beta_hml",weight=0)

,factor,risk_premium,t_statistic,t_statistic_newey_west
0,Intercept,0.398215,1.553175,1.571744
1,beta_hml,0.130066,0.489303,0.465590
2,beta_mkt,-0.037324,-0.081560,-0.077571
3,beta_smb,0.520175,1.371739,1.558102


### 练习二

1. 使用基于公司规模的加权最小二乘回归计算风险溢价（Hou、Xue、Zhang，2020）
2. 不进行权重调整，但删除每月规模最小的20%的公司
3. 比较文章主体结果和上面两种方式的三种方式之间计算的风险溢价的不同

In [26]:
data_fama_macbeth=data_fama_macbeth_copy

In [27]:
csr_tsagg(data_fama_macbeth,formula="ret_excess_lead ~ beta + log_mktcap + bm",weight=1)

,factor,risk_premium,t_statistic,t_statistic_newey_west
0,Intercept,1.076361,3.973785,3.953525
1,beta,-0.032167,-0.230120,-0.222392
2,bm,0.017898,0.168491,0.148133
3,log_mktcap,-0.057572,-2.382217,-2.311482


删除每月市值后20%的数据，进行FM回归


In [28]:
data_fama_macbeth = (data_fama_macbeth
 .groupby("month",group_keys=False)
 .apply(lambda x: x.assign(larger=np.where(x["log_mktcap"]>x["log_mktcap"].quantile(0.20),
                                    1,0)))
 .query("larger == 1")) ## 将市值大于20%分位数的股票赋值为1并且筛选出来

In [29]:
csr_tsagg(data_fama_macbeth,formula="ret_excess_lead ~ beta + log_mktcap + bm",weight=0)

,factor,risk_premium,t_statistic,t_statistic_newey_west
0,Intercept,0.878339,3.536886,3.273242
1,beta,0.003170,0.028201,0.026029
2,bm,0.137847,1.849944,1.690306
3,log_mktcap,-0.049606,-1.569433,-1.526183


##### 结论：
1. 市值加权最小二乘回归: 账面市值比变得不显著了；规模仍然负显著，但是系数绝对值变小了一半
2. 删除每月市值后20%的FM回归: 账面市值比显著性降低；规模变得不显著
3. 这表明账面市值比和规模溢价很大程度上来源于小公司

### 练习三
1. 滚动窗口计算因子暴露的估计值，已有CAPM beta,需要补充计算市值和价值因子的beta
2. 计算风险溢价

In [30]:
data_fama_macbeth = data_fama_macbeth_copy

##### 计算市值和价值因子的beta

In [31]:
crsp_monthly_for_beta = (crsp_monthly
                         .get(["permno","month","ret_excess"])
                         .merge(factors_ff3_monthly,
                                how="left",
                                on=["month"])
                         .drop("rf",axis=1))

数据量很大，需要使用多线程

In [32]:
from joblib import Parallel, delayed, cpu_count
n_cores = cpu_count() - 1

In [33]:
## 筛选时期大于等于60个月（不进行这一步的话后面多线程回归会报错）
valid_permnos = (crsp_monthly_for_beta
                 .groupby("permno")['permno']
                 .count()
                 .reset_index(name="counts")
                 .query("counts >= 60"))

In [34]:
## 筛选股票
crsp_monthly_for_beta_groups = (crsp_monthly_for_beta
                                .merge(valid_permnos, 
                                        how="inner", 
                                        on="permno")
                                .dropna()
                                .groupby(["permno"],group_keys = False))

In [35]:
beta_smb = (pd.concat(Parallel(n_jobs=n_cores)
                          (delayed(roll_capm_estimation)(data[1],window_size = 60, min_obs = 48,formula="ret_excess~smb",params="smb",asset_type="permno")
                           for data in crsp_monthly_for_beta_groups)
                          ) ## 对于分组数据，其中的每个元素为元组，元组的第一个元素为分组依据（这里为股票代码），第二个元素是数据框
                .dropna()
                .rename(columns={"beta": "beta_smb"})
                )

In [36]:
beta_hml = (pd.concat(Parallel(n_jobs=n_cores)
                          (delayed(roll_capm_estimation)(data[1],window_size = 60, min_obs = 48,formula="ret_excess~hml",params="hml",asset_type="permno")
                           for data in crsp_monthly_for_beta_groups)
                          ) 
                .dropna()
                .rename(columns={"beta": "beta_hml"})
                )

In [37]:
## 合并风险暴露因子
beta_factors=(beta
              .rename(columns={"beta_monthly":"beta_mkt"})
              .merge(beta_smb,
                     how="left",
                     on=["month","permno"])
              .merge(beta_hml,
                     how="left", 
                     on=["month","permno"])
              .assign(month = lambda x: x["month"]+pd.DateOffset(months=1))
              .dropna())

In [38]:
data_fama_macbeth  = (data_fama_macbeth
                      .get(["month","permno","ret_excess_lead"])
                      .merge(beta_factors,
                             how="left",
                             on=["month","permno"])
                      .dropna())

##### 进行FM回归

In [39]:
csr_tsagg(data_fama_macbeth,formula="ret_excess_lead ~ beta_mkt + beta_smb + beta_hml",weight=0)

,factor,risk_premium,t_statistic,t_statistic_newey_west
0,Intercept,0.758599,6.274932,5.287137
1,beta_hml,0.144110,2.299934,2.073773
2,beta_mkt,-0.033258,-0.333832,-0.298595
3,beta_smb,0.173052,2.319871,2.100394


##### 结论
1. 市值和价值因子的溢价都显著，市场因子的溢价不显著
2. 与文中结果一致